In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load Data

In [3]:
from utils import load_training_data, mute_stdout, \
    StandardScaler, accuracy_score, \
    cross_val_score,train_test_split, rasterize

x_full, y_full = load_training_data('training_data')

x_train, x_test, y_train, y_test = train_test_split(
    x_full, y_full.values, test_size=.25
)

print(len(x_train), len(y_train), len(x_test), len(y_test))

750 750 250 250


# Rasterization only

In [4]:
raster_size = 16

x_train_r = [rasterize(x.values, raster_size) for x in x_train]
x_train_r = np.stack(x_train_r)
y_train = y_train.copy()

x_test_r = [rasterize(x.values, raster_size) for x in x_test]
x_test_r = np.stack(x_test_r)
y_test = y_test.copy()

In [5]:
x_train_flat = x_train_r.reshape(-1, raster_size ** 2)
x_test_flat = x_test_r.reshape(-1, raster_size ** 2)

In [6]:
scale = StandardScaler()
scale.fit(x_train_flat)

x_train_scale = scale.transform(x_train_flat)
x_test_scale = scale.transform(x_test_flat)

In [7]:
from utils import cross_val_score
from utils import KNN

# clf = SVC()
clf = KNN(5)
clf.fit(x_train_flat, y_train)

accuracy_score(y_test, clf.predict(x_test_flat))

0.708

# Rasterization and PCA

In [8]:
from utils import PCA

pca = PCA(64)


# Fit on train only 
pca.fit(x_train_scale)

# Transform both
x_train_pca = pca.transform(x_train_scale)
x_test_pca = pca.transform(x_test_scale)

x_train_pca.shape, x_test_pca.shape

((750, 64), (250, 64))

In [9]:
clf = KNN(1)
cv_score = cross_val_score(
                clf, 
                x_train_pca, y_train, 
                cv = 4)
print("Training CV accuracy: ", cv_score.mean())
clf.fit(x_train_pca, y_train)
y_pred = clf.predict(x_test_pca)
print("Test accuracy: ", accuracy_score(y_test, y_pred))

Training CV accuracy:  0.8917112
Test accuracy:  0.896
